<a href="https://colab.research.google.com/github/bhanu1234566789/ad--nlp/blob/main/at_1(excelr).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract pillow pdfplumber spacy
!python -m spacy download en_core_web_sm
!apt-get update
!apt-get install tesseract-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download

In [ ]:
import pytesseract
from PIL import Image
import pdfplumber
import spacy
import re
import os
import io
import json

In [ ]:
def process_resume_text(resume_text):
    """
    Processes the resume text using NLP to extract key information.

    Args:
        resume_text (str): Text extracted from the resume.

    Returns:
        dict: A dictionary containing extracted resume details.
    """
    if not resume_text:
        return {} # Return empty dict if no text

    nlp_doc = nlp(resume_text)
    extracted_data = {}

    # --- Basic Information Extraction ---
    name = None
    emails = set()
    phone_numbers = set()
    skills_keywords = [ # Expand this list!
        "Python", "Java", "C++", "JavaScript", "SQL", "NLP", "Machine Learning", "Deep Learning", "Data Analysis",
        "Project Management", "Communication", "Leadership", "Teamwork", "Problem-solving", "Microsoft Office", "Cloud Computing",
        "AWS", "Azure", "GCP", "Docker", "Kubernetes", "React", "Angular", "Node.js" # Example skills - customize!
    ]
    extracted_skills = set()
    experience_keywords = ["Experience", "Work History", "Professional Experience", "Employment History"]
    education_keywords = ["Education", "Qualifications", "Academic Background", "Degrees"]
    projects_keywords = ["Projects", "Personal Projects", "Technical Projects"]

    # --- Name Extraction (Simple Heuristic - improve with NER if needed) ---
    for ent in nlp_doc.ents:
        if ent.label_ == "PERSON" and name is None: # Take the first PERSON entity as name (can be refined)
            name = ent.text.strip()
            break

    # --- Email and Phone Number Extraction (using Regex) ---
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    phone_pattern = r"(\+\d{1,3})?[\s.-]?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}" # Basic pattern - improve for international formats

    emails.update(re.findall(email_pattern, resume_text))
    phone_numbers.update(re.findall(phone_pattern, resume_text))

    # --- Skill Extraction (Keyword based) ---
    for skill in skills_keywords:
        if re.search(r"\b" + re.escape(skill) + r"\b", resume_text, re.IGNORECASE): # Word boundary and case-insensitive search
            extracted_skills.add(skill)

    # --- Section Segmentation (Keyword based - improve with more sophisticated NLP) ---
    experience_section = ""
    education_section = ""
    projects_section = ""

    for keyword in experience_keywords:
        match = re.search(keyword, resume_text, re.IGNORECASE)
        if match:
            start_index = match.start()
            # Heuristic to find end of section - look for next section keyword or end of doc
            end_index = len(resume_text)
            for next_keyword_list in [education_keywords, projects_keywords, experience_keywords]: # Check for next section keywords
                for next_keyword in next_keyword_list:
                    next_match = re.search(next_keyword, resume_text[start_index+len(keyword):], re.IGNORECASE)
                    if next_match:
                        end_index = min(end_index, start_index + len(keyword) + next_match.start())
                        break # Break inner loop if found
                if end_index != len(resume_text):
                    break # Break outer loop if end index is updated
            experience_section = resume_text[start_index:end_index].strip()
            break # Only take the first matched experience section keyword

    # (Similar logic for education_section and projects_section - you can copy and adapt the loop)
    for keyword in education_keywords:
        match = re.search(keyword, resume_text, re.IGNORECASE)
        if match:
            start_index = match.start()
            end_index = len(resume_text)
            for next_keyword_list in [experience_keywords, projects_keywords, education_keywords]:
                for next_keyword in next_keyword_list:
                    next_match = re.search(next_keyword, resume_text[start_index+len(keyword):], re.IGNORECASE)
                    if next_match:
                        end_index = min(end_index, start_index + len(keyword) + next_match.start())
                        break
                if end_index != len(resume_text):
                    break
            education_section = resume_text[start_index:end_index].strip()
            break

    for keyword in projects_keywords:
        match = re.search(keyword, resume_text, re.IGNORECASE)
        if match:
            start_index = match.start()
            end_index = len(resume_text)
            for next_keyword_list in [experience_keywords, education_keywords, projects_keywords]:
                for next_keyword in next_keyword_list:
                    next_match = re.search(next_keyword, resume_text[start_index+len(keyword):], re.IGNORECASE)
                    if next_match:
                        end_index = min(end_index, start_index + len(keyword) + next_match.start())
                        break
                if end_index != len(resume_text):
                    break
            projects_section = resume_text[start_index:end_index].strip()
            break


    extracted_data['name'] = name
    extracted_data['emails'] = list(emails)
    extracted_data['phone_numbers'] = list(phone_numbers)
    extracted_data['skills'] = list(extracted_skills)
    extracted_data['experience_section'] = experience_section
    extracted_data['education_section'] = education_section
    extracted_data['projects_section'] = projects_section

    return extracted_data

In [ ]:
def main_resume_extraction(resume_file_path):
    """
    Main function to perform resume extraction and display results.

    Args:
        resume_file_path (str): Path to the resume file to process.
    """
    extracted_text = extract_text_from_resume(resume_file_path)

    if extracted_text:
        print("\n--- Extracted Text ---\n")
        print(extracted_text[:500] + "..." if len(extracted_text) > 500 else extracted_text) # Print first 500 chars

        extracted_data = process_resume_text(extracted_text)

        print("\n--- Extracted Data (Structured) ---\n")
        print(json.dumps(extracted_data, indent=4)) # Nicely formatted JSON output

    else:
        print(f"Could not extract text from: {resume_file_path}")

In [ ]:
resume_file = "C:/Users/bhanu/OneDrive/Desktop/images.jpeg"
main_resume_extraction(resume_file)

Processing file: C:/Users/bhanu/OneDrive/Desktop/images.jpeg
